<a href="https://colab.research.google.com/github/ahed-jneed/Deep-Learning-Project/blob/main/Leopard_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
from fastbook import *
from fastai.vision.widgets import *

# Downloaded the URLs

In [ ]:
results = search_images_ddg('african leopard')
len(results), results[0]

In [ ]:
dest = 'images/african.jpg'
download_url(results[0], dest)
im = Image.open(dest)
im.thumbnail((128,128))
im

# Make a single File for each type

In [ ]:
leopard_types = 'african','black','teddy'
path = Path('leopard')

In [ ]:
if not path.exists():
    path.mkdir()
    for o in leopard_types:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_ddg(f'{o} leopard')
        download_images(dest, urls=results)

In [ ]:
fns = get_image_files(path)
fns

In [ ]:
failed = verify_images(fns)
failed

In [ ]:
failed.map(Path.unlink);

## From Data to DataLoaders

In [ ]:
leopards = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [ ]:
dls = leopards.dataloaders(path)

In [ ]:
dls.valid.show_batch(max_n=20, nrows=2)

One thing to be aware of in this process: models can only reflect the data used to train them. And the world is full of biased data, which ends up reflected in, for example, DuckDuckGo Image Search (which we used to create our dataset).
With this as our training data, you would end up not with a a balck leopard detector, but an Actor in a movie dressed as a black panther detector! Be sure to think carefully about the types of data that you might expect to see in practice in your application, and check carefully to ensure that all these types are reflected in your model's source data. 

In [ ]:
leopards = leopards.new(item_tfms = Resize(128, ResizeMethod.Squish))
dls = leopards.dataloaders(path)
dls.valid.show_batch(max_n = 4, nrows = 1)

By default Resize crops the images to fit a square shape of the size requested, using the full width or height. This can result in losing some important details. Alternatively, you can ask fastai to pad the images with zeros (black), or squish/stretch them:

In [ ]:
leopards = leopards.new(item_tfms = Resize(128, ResizeMethod.Pad, pad_mode = 'zeros' ))
dls = leopards.dataloaders(path)
dls.valid.show_batch(max_n = 4, nrows = 1)

In [ ]:
leopards = leopards.new(item_tfms =   RandomResizedCrop(128, min_scale = 0.3))
dls = leopards.dataloaders(path)
dls.train.show_batch(max_n = 4, nrows = 1, unique = True)

In [ ]:
leopards = leopards.new(item_tfms = Resize(128), batch_tfms = aug_transforms(mult = 2))
dls = leopards.dataloaders(path)
dls.train.show_batch(max_n = 8, nrows = 2, unique = True)

All of these approaches seem somewhat wasteful, or problematic. If we squish or stretch the images they end up as unrealistic shapes, leading to a model that learns that things look different to how they actually are, which we would expect to result in lower accuracy.

Instead, what inormally do in practice is to randomly select part of the image, and crop to just that part. On each epoch (which is one complete pass through all of our images in the dataset) we randomly select a different part of each image. This means that our model can learn to focus on, and recognize, different features in our images. It also reflects how images work in the real world: different photos of the same thing may be framed in slightly different ways.

## Training our Model, and Using It to Clean our Data

We don't have a lot of data for our problem (150 pictures of each sort of bear at most), so to train our model, we'll use RandomResizedCrop with an image size of 224 px, which is fairly standard for image classification, and default aug_transforms:

In [ ]:
leopards = leopards.new(
    item_tfms = RandomResizedCrop(224, min_scale = 0.5),
    batch_tfms = aug_transforms()
)
dls = leopards.dataloaders(path)

In [ ]:
learn = cnn_learner(dls, resnet18, metrics = error_rate)
learn.fine_tune(4)

Now let's see whether the mistakes the model is making are mainly thinking that blackes are teddies (that would be bad for safety!), or that african are black leopards, or something else. To visualize this, we can create a confusion matrix:

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(5, nrows = 1)

## Let's clean our images
as we can see the Model has predict the teddy black leopard as black leopard and there are many images are not the right place

In [ ]:
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
for idx in cleaner.delete(): cleaner.fns[idx].unlink()


In [ ]:
for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)

In [ ]:
learn.export()

In [ ]:
path = Path()
path.ls(file_exts='.pkl')

In [ ]:
learn_inf = load_learner(path/'export.pkl')

When we're doing inference, we're generally just getting predictions for one image at a time. To do this, pass a filename to predict:

In [ ]:
learn_inf.predict('/content/images/african.jpg')

This has returned three things: the predicted category in the same format you originally provided (in this case that's a string), the index of the predicted category, and the probabilities of each category.

In [ ]:
learn_inf.dls.vocab

## Creating a Notebook App from the Model

In [ ]:
btn_upload = widgets.FileUpload()
btn_upload

In [ ]:
#btn_upload = SimpleNamespace(data = ['/content/imags.jpg'])

In [ ]:
img = PILImage.create(btn_upload.data[-1])

We can use an Output widget to display it:

In [ ]:
out_pl = widgets.Output()
out_pl.clear_output()
with out_pl: display(img.to_thumb(128,128))
out_pl

In [ ]:
pred,pred_idx,probs = learn_inf.predict(img)

and use a Label to display them:

In [ ]:
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
lbl_pred

We'll need a button to do the classification. It looks exactly like the upload button:

In [ ]:
btn_run = widgets.Button(description='Classify')
btn_run

We'll also need a click event handler; that is, a function that will be called when it's pressed. We can just copy over the lines of code from above:

In [ ]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)

In [ ]:
#Putting back btn_upload to a widget for next cell
btn_upload = widgets.FileUpload()

In [ ]:
#hide_output
VBox([widgets.Label('Select your leopard!'), 
      btn_upload, btn_run, out_pl, lbl_pred])